## The dynamics of the temperature of a planet:

### A zero dimentional global energy balance model including ice albedo feedback

Placeholder cell for a description of the energy balance model and the albedo feedback effect
(you can add equations using the LaTex syntax. For e.g. $\frac{dT}{dt}$)

### Simulating the temperature dynamics by solving the energy balance model

In [1]:
# Import relevant packages

import numpy as np
from scipy.integrate import solve_ivp
from bokeh.io import show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row, column
from bokeh.palettes import Colorblind
output_notebook()

Loading BokehJS ...

Placeholder cell to describe what the physical parameters mean

In [2]:
# Define physical parameters

S0 = 1366.0 / 4 # W/m^2
alpha = 2.0
beta = 5.0
T_init = 2.0
emissivity = 0.6
sigma = 5.67e-8

In [3]:
# Define simulation parameters

timesteps = 500  # Number of time steps
timepoints = np.linspace(0, 10, timesteps)
tempsteps = 350  # Number of temperature steps
temperature = np.linspace(-150, 200, tempsteps)
albedo = np.zeros(tempsteps)
rad_incoming = np.zeros(tempsteps)
rad_outgoing = np.zeros(tempsteps)

In [4]:
# Model definitions

# Function defining the energy balance model
def EnergyBalanceEqn(t, y):
    T = y
    rin = IncomingRad(T, noise=True)
    rout = OutgoingRad(T)
    dTdt = [rin - rout]
    return dTdt

# Function calculating albedo
def CalcAlbedo(T):
    a_max = 0.9
    a_min = 0.1
    T0 = 255 - 273
    A = (a_max - a_min) / 2
    B = (a_max + a_min) / 2
    return A * np.tanh(alpha * (T / T0)) + B

# Function calculating incoming radiation*(1 - albedo) (with or without stochastic noise)
def IncomingRad(T, noise):
    if noise:  # including stochastic noise
        return S0 * (1.0 - CalcAlbedo(T) + beta * np.random.randn())
    else:  # not including stochastic noise
        return S0 * (1.0 - CalcAlbedo(T))

# Function calculating outgoing radiation
def OutgoingRad(T):
    return emissivity * sigma * (T + 273)**4

In [5]:
# Solve the energy balance model including albedo feedback for 4 different scenarios

timelist = []
templist = []
for i in range(4):
    sol = solve_ivp(EnergyBalanceEqn, (0, 10), [T_init], method='RK45', t_eval=timepoints)
    timelist.append(sol.t)
    templist.append(sol.y[0, :])

In [6]:
# Calculate albedo and radiation for given tempeature range (no stochastic noise)

albedo = CalcAlbedo(temperature)
rad_incoming = IncomingRad(temperature, noise=False)
rad_outgoing = OutgoingRad(temperature)
rad_diff = rad_incoming - rad_outgoing

In [7]:
# Plot results

# Figure 1
p1 = figure(plot_width=900, plot_height=400, x_axis_label='Time', y_axis_label='Temperature',
           title='Zero-dimensional energy balance model')
p1.multi_line(timelist, templist, color=Colorblind[4][:])
# p1.toolbar.logo = None
p1.toolbar_location = None

# Figure 2
p2 = figure(plot_width=300, plot_height=400, x_axis_label='Temperature', y_axis_label='Albedo',
           title='S0 = ' + str(S0) + ' W/m^2')
p2.line(temperature, albedo, color=Colorblind[3][0])
# p2.toolbar.logo = None
p2.toolbar_location = None

p3 = figure(plot_width=300, plot_height=400, x_axis_label='Temperature', y_axis_label='Radiation')
p3.line(temperature, rad_incoming, color=Colorblind[6][5], legend='incoming')
p3.line(temperature, rad_outgoing, color=Colorblind[6][0], legend='outgoing')
p3.legend.location = 'top_left'  
# p3.toolbar.logo = None
p3.toolbar_location = None

p4 = figure(plot_width=300, plot_height=400, x_axis_label='Temperature', y_axis_label='Radiation',
           title='dT/dt = incoming-outgoing')
p4.line(temperature, rad_diff, color=Colorblind[8][5])
p4.line(temperature, np.zeros(tempsteps), color=Colorblind[8][7])
# p4.toolbar.logo = None
p4.toolbar_location = None

# Display plots
show(column(p1, row(p2, p3, p4)))